# Machine Learning

In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
import os
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [ ]:
df_train=pd.read_csv("./../data/application_train.csv",encoding="utf-8",index_col=0)
df_test=pd.read_csv("./../data/application_test.csv",encoding="utf-8",index_col=0)
df = pd.concat([df_train,df_test])
df_safe = pd.concat([df_train,df_test])


In [ ]:
# comparaison entre df et df sans les nan

df_salvage_drop=df.dropna()
' '.join(['ratio:' , str(df_salvage_drop.shape[0]) ,'/' , str(df.shape[0]), '=' , str(round(df_salvage_drop.shape[0]/df.shape[0]*100,2)), '%'])

In [ ]:
# liste les valeurs par default de chaque clmn

def one_line_try(serie):
    try:
        return float(serie.mean())
    except:
        return float(np.nan)
    
default_value_per_clmn = {clmn : one_line_try(df_salvage_drop[clmn]) for clmn in df_salvage_drop}

default_value_per_clmn

In [ ]:
# applique ces valeur aux clmn

for clmn in df:
    df[clmn]= [A if str(A) != 'nan' else default_value_per_clmn[clmn] for A in df[clmn]]
df_cute_drop = df.dropna()
' '.join(['ratio:' , str(df_cute_drop.shape[0]) ,'/' , str(df.shape[0]), '=' ,str(round(df_cute_drop.shape[0]/df.shape[0]*100,2)), '%'])



In [ ]:
del df_cute_drop
del df_salvage_drop
df = df.dropna()

In [ ]:
colonnes=[colonne for colonne in df]
types_colonnes=list(map(lambda x : {x : list(set(df[x].map(lambda x : str(type(x)))))} , colonnes))
colonnes_str=list(filter(lambda x : "<class 'str'>" in x[list(x)[0]], types_colonnes))

def vectorisation(col):
    voc=list(set(list(col)))
    val={voc[i]:i for i in range(len(voc))}
    return col.map(lambda x : val[x])



for i in [list(i)[0] for i in colonnes_str]:
    df[i]=vectorisation(df[i])

In [ ]:
# liste les valeurs par default de chaque clmn

def one_line_try(serie):
    try:
        return float(serie.mean())
    except:
        return float(np.nan)
    
default_value_per_clmn = {clmn : one_line_try(df[clmn]) for clmn in df}

df = df_safe


    
for clmn in df:
    df[clmn]= [A if str(A) != 'nan' else default_value_per_clmn[clmn] for A in df[clmn]]
df_cute_drop = df.dropna()
' '.join(['ratio:' , str(df_cute_drop.shape[0]) ,'/' , str(df.shape[0]), '=' ,str(round(df_cute_drop.shape[0]/df.shape[0]*100,2)), '%'])

In [ ]:
default_value_per_clmn

In [ ]:
df

In [ ]:
del df_cute_drop
df = df.dropna()

colonnes=[colonne for colonne in df]
types_colonnes=list(map(lambda x : {x : list(set(df[x].map(lambda x : str(type(x)))))} , colonnes))
colonnes_str=list(filter(lambda x : "<class 'str'>" in x[list(x)[0]], types_colonnes))

def vectorisation(col):
    voc=list(set(list(col)))
    val={voc[i]:i for i in range(len(voc))}
    return col.map(lambda x : val[x])

for i in [list(i)[0] for i in colonnes_str]:
    df[i]=vectorisation(df[i])

In [ ]:
df

In [ ]:
df_true = df[df['TARGET'] == 1]
df_false = df[df['TARGET'] != 1]
maxi_val = min([df_false.shape[0], df_true.shape[0]])
maxsize = maxi_val*2
print('whith parity: {} / {}'.format(maxsize,df.shape[0],round(maxsize/df.shape[0],2)))
df = pd.concat([df_false.iloc[:maxi_val],df_true.iloc[:maxi_val]])

In [ ]:
df['OCCUPATION_TYPE'].value_counts()

In [ ]:
colonnes=[colonne for colonne in df]
list(map(lambda x : {x : list(set(df[x].map(lambda x : str(type(x)))))} , colonnes))

In [ ]:
# matrice de correlation

total_corr = df.corr()
total_corr

In [ ]:
# liste les meuilleurs correlations entre les clmns

def get_coor(mat, corr = .75):
    clmn_lst = mat.columns
    exeption = [] 
    val = 0
    for clmn in mat:
        cmd = lambda idx, val: [print("{} -> {} : ".format(clmn, clmn_lst[idx]).ljust(80, ' ') + str(val)),  exeption.append(val)]
        [cmd(idx, val) for idx, val in enumerate(mat[clmn]) if val > corr and val != 1 and val not in exeption]

        

get_coor(total_corr, .99)

In [ ]:
# liste les meuilleurs corelations par rapport a une clmn

def get_clmn_corr(mat, clmn, corr = 0.1):
    serie = mat[clmn]
    clmn_lst = list(serie.keys())
    cmd = lambda idx, val: print("{} -> {} : ".format(clmn, clmn_lst[idx]).ljust(80, ' ') + str(val))
    [cmd(idx, val) for idx, val in enumerate(mat[clmn]) if abs(val) > corr and val != 1]
    
get_clmn_corr(total_corr, 'TARGET', 0.05)

In [ ]:
#set(df_train['FONDKAPREMONT_MODE'].fillna("").map(type))

In [ ]:
#sorted(list(df.corr()['TARGET']))

In [ ]:
from machinelearning import prediction


file=open("machinelearning.py","r",encoding="utf-8")
#print(file.read())
file.close()
del file

In [ ]:
# harmonise les types
df = df.applymap(float)
df['TARGET'] = df['TARGET'].apply(int)

In [ ]:
prediction(df,models=[{"modèle":LinearSVC,"paramètres":{"random_state":44}},
                       {"modèle":RandomForestClassifier,"paramètres":{"n_estimators":750,"random_state":44}},
                       {"modèle":GradientBoostingClassifier,"paramètres":{"random_state":44}},
                       {"modèle":LogisticRegression,"paramètres":{"random_state":44}},
                      {"modèle":XGBClassifier,"paramètres":{}}
                     ])

#  Let's check models details

In [5]:
fichiers=os.listdir("./../")
fichiers=list(filter(lambda x : x[-5:]=='.html',fichiers))

In [6]:
from IPython.core.display import display, HTML
for fichier in fichiers:
    print(fichier[:-5],':')
    f=open("./../"+fichier,'r')
    g=f.read()
    f.close()
    
    display(HTML(g))
    f=open("./../"+fichier[:-4]+'txt','r')
    g=f.read()
    f.close()
    print(g,'\n\n')

GradientBoostingClassifier :


Weight,Feature
0.0671 ± 0.2392,x108
0.0583 ± 0.1926,x105
0.0494 ± 0.1704,x45
0.0421 ± 0.2032,x48
0.0367 ± 0.2615,x97
0.0294 ± 0.1379,x86
0.0280 ± 0.1740,x21
0.0262 ± 0.2998,x34
0.0240 ± 0.1679,x55
0.0228 ± 0.1370,x42


              precision    recall  f1-score   support

           0       0.87      0.75      0.81      4971
           1       0.78      0.89      0.83      4959

    accuracy                           0.82      9930
   macro avg       0.83      0.82      0.82      9930
weighted avg       0.83      0.82      0.82      9930
 


LinearSVC :


              precision    recall  f1-score   support

           0       0.79      0.08      0.15      4971
           1       0.52      0.98      0.67      4959

    accuracy                           0.53      9930
   macro avg       0.65      0.53      0.41      9930
weighted avg       0.65      0.53      0.41      9930
 


LogisticRegression :


              precision    recall  f1-score   support

           0       0.57      0.93      0.71      4971
           1       0.80      0.30      0.44      4959

    accuracy                           0.61      9930
   macro avg       0.69      0.61      0.57      9930
weighted avg       0.69      0.61      0.57      9930
 


RandomForestClassifier :


Weight,Feature
0.0162 ± 0.0671,x86
0.0145 ± 0.0577,x48
0.0141 ± 0.0565,x105
0.0138 ± 0.0561,x108
0.0128 ± 0.0568,x45
0.0126 ± 0.0498,x97
0.0121 ± 0.0461,x21
0.0117 ± 0.0498,x51
0.0115 ± 0.0424,x42
0.0114 ± 0.0492,x20


              precision    recall  f1-score   support

           0       0.88      0.91      0.90      4971
           1       0.91      0.88      0.89      4959

    accuracy                           0.89      9930
   macro avg       0.90      0.89      0.89      9930
weighted avg       0.90      0.89      0.89      9930
 


XGBClassifier :


Weight,Feature
0.0686,f108
0.0643,f105
0.0369,f86
0.0306,f119
0.0267,f48
0.0235,f76
0.0229,f18
0.0222,f90
0.0218,f51
0.0206,f10


              precision    recall  f1-score   support

           0       0.92      0.89      0.90      4971
           1       0.89      0.92      0.91      4959

    accuracy                           0.90      9930
   macro avg       0.91      0.90      0.90      9930
weighted avg       0.91      0.90      0.90      9930
 




In [ ]:
#Attention ne pas lancer si mlflow ne vous intéresse pas
os.chdir("./../")
import webbrowser
webbrowser.open('http://127.0.0.1:5000/', new=2)
os.popen('mlflow ui').read()